# 오답 행렬 초기화
## 처음 1회만 시행한 후, 신변에 문제가 생기지 않는 이상, 건드리지 말 것!

In [ ]:
# 43 by 43의 2차원 리스트 생성
matrix = [[0 for _ in range(44)] for _ in range(44)] # 나중에 쓸 오답 행렬이 된다.

# 필요 라이브러리 임포트

In [ ]:
! pip install konlpy
! pip install transformers
! pip install kiwipiepy

In [ ]:
# 임포트 해야 하는 라이브러리 모음

import numpy as np                                      # 넘파이: 스마트한 배열
import pandas as pd                                     # 판다스: 데이터 관련 연산
import matplotlib.pyplot as plt                         # 플롯: 각종 그래프 그리기
import seaborn as sns                                   # 씨본: 히스토그램 그리기
from sklearn.preprocessing import MinMaxScaler          # 사이킷런에서 "최대 최소 정규화" 모듈을 가져오기
from sklearn.preprocessing import StandardScaler        # 사이킷런에서 "표준 정규화" 모듈을 가져오기
from statsmodels.graphics.mosaicplot import mosaic      # 모자이크 플롯 그리기
from scipy import stats as sts                          # 싸이파이: 통계, 공학 라이브러리

# 플롯에 한글 폰트 깨짐 방지(세션 재기동 필요)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

from matplotlib import font_manager, rc
plt.rc('font', family='NanumBarunGothic')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

# 데이터 불러오기 및 기본 정제
## 기존에 임의로 나누어져 있던 트레인 테스트 합쳐서 하나로 만들기

In [ ]:
data_mo = pd.read_csv('merge_모든데이터.csv')
data_test = pd.read_csv('test_merge_모든데이터.csv')
job_label = pd.read_csv('data_final_job_label.csv')

# data_test의 student167 열 삭제
data_test = data_test.drop('student167', axis=1)

# data_mo와 data_test를 세로 방향으로 콘캐트
data = pd.concat([data_mo, data_test], axis=0)

# student_idx 열을 뒤에서 4자리만 취한 후, 정수형으로 변경
data['student_idx'] = data['student_idx'].str[-4:].astype(int)

# 직업을 ID 값으로 바꾸기
merged_data = data.merge(job_label, left_on='priority1', right_on='job_name', how='left')
data['priority1'] = merged_data['ID']

merged_data = data.merge(job_label, left_on='priority2', right_on='job_name', how='left')
data['priority2'] = merged_data['ID']

merged_data = data.merge(job_label, left_on='priority3', right_on='job_name', how='left')
data['priority3'] = merged_data['ID']

# 삭제할 열을 정의하고, 삭제하기
columns_to_drop = ['summary1', 'summary2', 'counsellor_idx',
                   'question1', 'question2', 'question3', 'question4', 'question5', 'question6']
data = data.drop(columns=columns_to_drop)

print("*" * 200)
display(data.head())
print("*" * 200)
print(data.info())
print("*" * 200)

# student_idx의 유니크한 값의 개수 확인
print(data['student_idx'].nunique())

********************************************************************************************************************************************************************************************************


,student_idx,priority1,priority2,priority3,combined_conversation
0,155,9,22,11,어서 오세요 [이모티콘] 방학인데 휴가는 다녀왔어요? 네! 어디로 다녀왔나요? 충주...
1,157,1,4,3,안녕하세요? 학생은 초등학교 6학년인가요? 네 여름방학 생활은 어떤가요? 쉴 수 있...
2,170,4,1,3,안녕하세요? 안녕하세요 오늘 상담을 진행할 카카오톡 상담사입니다 지금 상담할 수 있...
3,188,27,38,8,안녕하세요 안녕하세요 저는 오늘 상담을 맡게 될 카카오톡 상담사입니다 점심은 먹었나...
4,219,9,29,11,안녕하세요? 안녕하세요. 오늘 상담을 맡은 상담사입니다. 지금 상담 가능하세요? ...


********************************************************************************************************************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
Index: 5849 entries, 0 to 648
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   student_idx            5849 non-null   int64 
 1   priority1              5849 non-null   int64 
 2   priority2              5849 non-null   int64 
 3   priority3              5849 non-null   int64 
 4   combined_conversation  5849 non-null   object
dtypes: int64(4), object(1)
memory usage: 274.2+ KB
None
********************************************************************************************************************************************************************************************************
5849


# 일단 기본 모델로 해 보기
## --> Kiwi 파편화(보통명사, 고유명사 추출)
## --> KR-SBERT 오알 TF-IDF 벡터화
## --> SVD 차원 축소
## --> 시퀀셜 모형으로 다중 분류

## 키위, 버트

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix
from kiwipiepy import Kiwi
from collections import Counter
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# data에서 full_text, priority1 만 남기기
df = data[['combined_conversation', 'priority1']]

# 결과 출력
display(df.head(5))
print("*" * 200)

# KR-SBERT 모델 및 토크나이저 로드
model_name = "jhgan/ko-sbert-sts"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Kiwi 초기화
kiwi = Kiwi()

# 금지어 리스트 50
stopwords = ["생각", "진로", "직업", "학생", "상담", "관련", "관심", "학교", "감사", "대학",
             "시간", "공부", "분야", "학과", "학년", "선생", "이야기", "정보", "사람", "검사",
             "고민", "준비", "친구", "과목", "선택", "부분", "고등학교", "도움", "중요", "대학교",
             "성적", "진학", "전공", "정도", "안녕", "필요", "다양", "본인", "취업", "생활",
             "부모", "방법", "이모티콘", "경험", "이유", "안녕하세요", "내신", "방법", "경우", "얘기"]

# 의미 있는 키워드인지 검사
def is_meaningful_keyword(keyword):
    return len(keyword) >= 2 and not bool(re.search(r'\d|[!@#$%^&*(),.?":{}|<>]', keyword))

# 키워드 추출 함수 정의 (금지어 제외 및 의미 있는 키워드 필터링)
def extract_keywords(text, N=10):
    res = kiwi.tokenize(text)
    meishi_list = [r[0] for r in res if r[1] in ["NNG", "NNP"] and r[0] not in stopwords and is_meaningful_keyword(r[0])]
    keyword_counts = Counter(meishi_list)
    top_keywords = keyword_counts.most_common(N)
    return ' '.join([keyword for keyword, _ in top_keywords])

# 텍스트를 벡터로 변환하는 함수 정의
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# 데이터프레임의 텍스트에서 키워드 추출 후 벡터화
X = np.vstack([embed_text(extract_keywords(text, N=20)) for text in df['combined_conversation']])

# 레이블 원-핫 인코딩
y = df['priority1'].values
y = to_categorical(y)

## 키위, TF-IDF

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from kiwipiepy import Kiwi
from collections import Counter
import re

# data에서 full_text, priority1 만 남기기
df = data[['combined_conversation', 'priority1']]

# 결과 출력
display(df.head(5))
print("*" * 200)

# Kiwi 초기화
kiwi = Kiwi()

# 금지어 리스트 50
stopwords = ["생각", "진로", "직업", "학생", "상담", "관련", "관심", "학교", "감사", "대학",
             "시간", "공부", "분야", "학과", "학년", "선생", "이야기", "정보", "사람", "검사",
             "고민", "준비", "친구", "과목", "선택", "부분", "고등학교", "도움", "중요", "대학교",
             "성적", "진학", "전공", "정도", "안녕", "필요", "다양", "본인", "취업", "생활",
             "부모", "방법", "이모티콘", "경험", "이유", "안녕하세요", "내신", "방법", "경우", "얘기"]

# 의미 있는 키워드인지 검사
def is_meaningful_keyword(keyword):
    return len(keyword) >= 2 and not bool(re.search(r'\d|[!@#$%^&*(),.?":{}|<>]', keyword))

# 키워드 추출 함수 정의 (금지어 제외 및 의미 있는 키워드 필터링)
def extract_keywords(text, N=10):
    res = kiwi.tokenize(text)
    meishi_list = [r[0] for r in res if r[1] in ["NNG", "NNP"] and r[0] not in stopwords and is_meaningful_keyword(r[0])]
    keyword_counts = Counter(meishi_list)
    top_keywords = keyword_counts.most_common(N)
    return ' '.join([keyword for keyword, _ in top_keywords])

# 키워드 추출(기본 N 값은 10)
df['keywords'] = df['combined_conversation'].apply(lambda x: extract_keywords(x, N=25))

# 데이터 전처리
X = df['keywords'].values
y = df['priority1'].values

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(X).toarray()

# 레이블 원-핫 인코딩
y = df['priority1'].values
y = to_categorical(y)

,combined_conversation,priority1
0,어서 오세요 [이모티콘] 방학인데 휴가는 다녀왔어요? 네! 어디로 다녀왔나요? 충주...,9
1,안녕하세요? 학생은 초등학교 6학년인가요? 네 여름방학 생활은 어떤가요? 쉴 수 있...,1
2,안녕하세요? 안녕하세요 오늘 상담을 진행할 카카오톡 상담사입니다 지금 상담할 수 있...,4
3,안녕하세요 안녕하세요 저는 오늘 상담을 맡게 될 카카오톡 상담사입니다 점심은 먹었나...,27
4,안녕하세요? 안녕하세요. 오늘 상담을 맡은 상담사입니다. 지금 상담 가능하세요? ...,9


********************************************************************************************************************************************************************************************************


<ipython-input-6-812b8a53fe1e>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['combined_conversation'].apply(lambda x: extract_keywords(x, N=25))


## 차원 축소 및 모델 구축

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# SVD를 사용하여 차원을 축소하는 함수 정의
def reduce_dimension(X, n_components = 100):
    svd = TruncatedSVD(n_components=n_components)
    X_reduced = svd.fit_transform(X)
    return X_reduced

# 데이터프레임의 텍스트에서 키워드 추출 후 벡터화된 데이터를 SVD로 차원 축소
X_reduced = reduce_dimension(X, n_components = 500)

# 결과 확인
print("Original shape:", X.shape)
print("Reduced shape:", X_reduced.shape)

Original shape: (5849, 6372)
Reduced shape: (5849, 500)


In [ ]:

from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import *
import keras

def do_model(k=10, show=False):
    '''
    모델을 실행하고, 결과를 보여주는 함수

    k: 반복 횟수
    show: 결과를 보여줄지 여부
    '''

    for i in range(k):

        # 반복 시작 알림
        print(f"반복 {i+1}번 시작")
        print( )

        # 데이터 분리
        X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.1)

        # 모델 구축
        model = Sequential()
        model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(y.shape[1], activation='softmax'))

        # 학습률 설정
        optimizer = Adam(learning_rate=0.001)

        # 조기종료 설정
        early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,
                                                       restore_best_weights=True)


        # 모델 컴파일
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # 모델 학습
        history = model.fit(X_train, y_train,
                            epochs=180, batch_size=256, validation_split=0.3,
                            callbacks=[early_stopping],
                            verbose = 0)

        # 모델 평가
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"모델의 정확도: {accuracy * 100:.2f} %")

        # 예측 결과
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_test, axis=1)

        # 오답 행렬의 빈출 오답은 서로 예측해도 정답으로 처리되도록, 정확도 처리 함수 변경
        def calculate_accuracy_2(y_true, y_pred):
            correct = 0
            total = len(y_true)
            for i in range(total):
                if y_true[i] == y_pred[i]:
                    correct += 1

                # IT 관련 < > 공학
                elif (y_true[i] == 1 and y_pred[i] == 4) or (y_true[i] == 4 and y_pred[i] == 1):
                    correct += 1

                # 공학 < > 이학
                elif (y_true[i] == 4 and y_pred[i] == 22) or (y_true[i] == 22 and y_pred[i] == 4):
                    correct += 1

                # IT 관련 < > 공학
                elif (y_true[i] == 1 and y_pred[i] == 22) or (y_true[i] == 22 and y_pred[i] == 1):
                    correct += 1

                # 디자인 < > 게임, 애니메이션
                elif (y_true[i] == 15 and y_pred[i] == 21) or (y_true[i] == 21 and y_pred[i] == 15):
                    correct += 1

                # 의료 전문 < > 의료 서비스
                elif (y_true[i] == 9 and y_pred[i] == 33) or (y_true[i] == 33 and y_pred[i] == 9):
                    correct += 1

                # 의료 전문 < > 이학
                elif (y_true[i] == 9 and y_pred[i] == 22) or (y_true[i] == 22 and y_pred[i] == 9):
                    correct += 1

                # 안전 < > 운동
                elif (y_true[i] == 7 and y_pred[i] == 37) or (y_true[i] == 37 and y_pred[i] == 7):
                    correct += 1

            return correct / total

        accuracy = calculate_accuracy_2(y_true, y_pred_classes)
        print(f"수정된 모델의 정확도: {accuracy * 100:.2f} %")

        # 틀린 예측만 추출
        incorrect_indices = np.where(y_true != y_pred_classes)[0]
        incorrect_true_labels = y_true[incorrect_indices]
        incorrect_pred_labels = y_pred_classes[incorrect_indices]

        # 각 클래스별로 틀린 예측을 카운트
        incorrect_counts = {}
        for true_label in np.unique(incorrect_true_labels):
            incorrect_preds = incorrect_pred_labels[incorrect_true_labels == true_label]
            incorrect_counter = Counter(incorrect_preds)
            most_common_incorrect = incorrect_counter.most_common(1)
            if most_common_incorrect:
                incorrect_counts[true_label] = most_common_incorrect[0]

        # 결과를 데이터프레임으로 변환
        incorrect_df = pd.DataFrame.from_dict(incorrect_counts, orient='index')

        # 인덱스를 새로운 열로 만들기
        incorrect_df.reset_index(inplace=True)
        incorrect_df.columns = ['정답', '가장 많은 오답', '오답 횟수']

        # incorrect_df의 각 행을 순회
        for index, row in incorrect_df.iterrows():
            true_label = row['정답']
            most_common_incorrect = row['가장 많은 오답']
            count = row['오답 횟수']
            matrix[true_label][most_common_incorrect] += count
        print("*" * 150)

    if show:
        #매트릭스를 히트맵으로 출력
        plt.figure(figsize=(16, 12))
        sns.heatmap(matrix, annot=True,
                    fmt='d', cmap='Blues',
                    annot_kws={"size": 10},
                    linewidths=0.5, linecolor='white')

        plt.xticks(fontsize=11, rotation=0)
        plt.yticks(fontsize=11, rotation=0)

        plt.xlabel('Predicted', fontsize=14)
        plt.ylabel('True', fontsize=14)
        plt.title('누적 오답행렬', fontsize=14)
        plt.show()

    return print('모든 과정 종료')

do_model(k = 2, show = False)

반복 1번 시작

19/19 [==============================] - 0s 2ms/step - loss: 2.4632 - accuracy: 0.4410
모델의 정확도: 44.10 %
19/19 [==============================] - 0s 1ms/step
수정된 모델의 정확도: 49.06 %
******************************************************************************************************************************************************
반복 2번 시작

19/19 [==============================] - 0s 2ms/step - loss: 2.5617 - accuracy: 0.4188
모델의 정확도: 41.88 %
19/19 [==============================] - 0s 1ms/step
수정된 모델의 정확도: 46.50 %
******************************************************************************************************************************************************
모든 과정 종료


In [ ]:

# # 혼동 행렬 계산
# conf_mat = confusion_matrix(y_true, y_pred_classes)

# plt.figure(figsize=(16, 12))
# sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.show()

# # 정확도 그래프 그리기. 범례와 축 제목 표시
# plt.plot(history.history['accuracy'], label='Training Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

# 오답 분석

In [ ]:
# # 틀린 예측만 추출
# incorrect_indices = np.where(y_true != y_pred_classes)[0]
# incorrect_true_labels = y_true[incorrect_indices]
# incorrect_pred_labels = y_pred_classes[incorrect_indices]

# # 각 클래스별로 틀린 예측을 카운트
# incorrect_counts = {}
# for true_label in np.unique(incorrect_true_labels):
#     incorrect_preds = incorrect_pred_labels[incorrect_true_labels == true_label]
#     incorrect_counter = Counter(incorrect_preds)
#     most_common_incorrect = incorrect_counter.most_common(1)
#     if most_common_incorrect:
#         incorrect_counts[true_label] = most_common_incorrect[0]

# # 결과를 데이터프레임으로 변환
# incorrect_df = pd.DataFrame.from_dict(incorrect_counts, orient='index')

# # 인덱스를 새로운 열로 만들기
# incorrect_df.reset_index(inplace=True)
# incorrect_df.columns = ['정답', '가장 많은 오답', '오답 횟수']

# # 결과 출력
# print("틀린 예측 중 가장 많이 틀린 예측 결과 테이블")
# print("*" * 50)
# display(incorrect_df.head(10))
# print("*" * 200)

# # incorrect_df의 각 행을 순회
# for index, row in incorrect_df.iterrows():
#     true_label = row['정답']
#     most_common_incorrect = row['가장 많은 오답']
#     count = row['오답 횟수']
#     matrix[true_label][most_common_incorrect] += count

# #매트릭스를 히트맵으로 출력
# plt.figure(figsize=(16, 12))
# sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('matrix')
# plt.show()